# Grounding Gemini with Vertex AI Agent Builder

## Overview

[Grounding in Vertex AI](https://cloud.google.com/vertex-ai/generative-ai/docs/grounding/overview) lets you use generative text models to generate content grounded in your own documents and data. This capability lets the model access information at runtime that goes beyond its training data. By grounding model responses in data stores within [Vertex AI Agent Builder](https://cloud.google.com/generative-ai-app-builder/docs/enterprise-search-introduction), LLMs that are grounded in data can produce more accurate, up-to-date, and relevant responses.

Grounding provides the following benefits:

- Reduces model hallucinations (instances where the model generates content that isn't factual)
- Anchors model responses to specific information, documents, and data sources
- Enhances the trustworthiness, accuracy, and applicability of the generated content

### Learning Objectives

In this notebook, you learn how to:

- Compare the results of ungrounded LLM responses with grounded LLM responses
- Create and use a data store in Vertex AI Search to ground responses in custom documents and data
- Generate LLM text and chat model responses grounded in Vertex AI Search results

This tutorial uses the following Google Cloud AI services and resources:

- Vertex AI
- Vertex AI Search and Conversation

The steps performed include:

- Configuring the LLM and prompt for various examples
- Sending example prompts to generative text and chat models in Vertex AI
- Setting up a data store in Vertex AI Search with your own data
- Sending example prompts with various levels of grounding (no grounding, data store grounding)

This notebook uses a specific kernel to include needed dependencies. Run the cell below to create it and then select `gemini_kernel` in the top right before moving forward.

In [ ]:
!cd ~/asl-ml-immersion && make gemini_kernel

In [ ]:
from google.cloud import aiplatform

print(aiplatform.__version__)

In [ ]:
PROJECT_ID = !gcloud config list --format 'value(core.project)'
PROJECT_ID = PROJECT_ID[0]
REGION = "us-central1"

### Import libraries

In [ ]:
from IPython.display import Markdown, display
from vertexai.preview.generative_models import (
    GenerationConfig,
    GenerativeModel,
    Tool,
    grounding,
)

Initialize the Gemini model:

In [ ]:
model = GenerativeModel("gemini-1.5-pro-001")

## Grounding with custom documents and data

In this example, you'll compare LLM responses with no grounding with responses that are grounded in the [results of a data store in Vertex AI Search](https://cloud.google.com/generative-ai-app-builder/docs/create-datastore-ingest). You'll ask a question about a GoogleSQL query to create an [object table in BigQuery](https://cloud.google.com/bigquery/docs/object-table-introduction).

### Creating a data store in Vertex AI Agent Builder

Follow the steps below to create a data store in Vertex AI Agent Builder with sample data. In this example, you'll use a website-based data store that contains content from the Google Cloud website, including documentation.

1. In the Google Cloud console, go to the [Agent Builder](https://console.cloud.google.com/gen-app-builder/engines?_ga=2.175627042.1438957058.1715191726-353778574.1715191726) page.
2. Click **Create App**.
3. In the Select app type pane, select **Search**.
4. If you are offered a choice of generic or media content, click **Generic**.
5. In order to create a website search app, make sure **Enterprise features** is turned on.
6. If you don't plan to use **Advanced LLM** features for this app, turn off the **Advanced LLM** features option.
7. In the **Your app name** field, enter a name for your app. Your app ID appears under the app name.
8. In the **External name** of your company or organization field, enter the company or organization name.
9. Select **global (Global)** as the location for your app, and then click Continue.
10. In the **Data** pane, click **Create new data store**.
11. In the Select a data source pane, select **Website Content**.
12. Make sure that **Advanced website indexing** is **turned off**.
13. In the **Specify the websites** for your data store pane, in the Sites to include field, enter `cloud.google.com/*` and then click Continue.
14. In the **Configure your data store** pane, enter a name for your data store, and then click **Create**.
15. On the **Data stores** page, select your new data store, and then click **Create**.

**Important: Once you've created a data store, obtain the Data Store ID from the ID column and input it below.**

In [ ]:
DATA_STORE_PROJECT_ID = PROJECT_ID  # @param {type:"string"}
DATA_STORE_REGION = "global"  # @param {type:"string"}
# Replace this with your data store ID from Vertex AI Search
DATA_STORE_ID = ""  # TODO: ENTER YOUR DATASTORE ID

Now you can ask a question about object tables in BigQuery and when to use them:

In [ ]:
PROMPT = "When to use an object table in BigQuery?"

### Text generation without grounding

Make a prediction request to the LLM with no grounding:

In [ ]:
response = model.generate_content(PROMPT)
print(f"Response from model without grounding:\n {response.text}")

### Text generation grounded in Vertex AI Search results

Now we can add the `tools` keyword arg with a grounding tool of `grounding.VertexAISearch()` to instruct the LLM to first perform a search within your custom data store, then construct an answer based on the relevant documents:

In [ ]:
if DATA_STORE_ID and DATA_STORE_REGION:
    # Use Vertex AI Search data store
    tool = Tool.from_retrieval(
        grounding.Retrieval(
            grounding.VertexAISearch(
                datastore=DATA_STORE_ID,
                project=PROJECT_ID,
                location="global",
            )
        )
    )
else:
    print("Please provide DATA_STORE_ID and DATA_STORE_REGION")

response = model.generate_content(PROMPT, tools=[tool])
print(f"Response from Model:\n{response.text}")
print(f"\n\nGrounding Metadata:\n{response.candidates[0].grounding_metadata}")

Note that the response without grounding only has limited information from the LLM about object tables in BigQuery that might not be accurate. Whereas the response that was grounded in Vertex AI Search results contains the most up to date information from the Google Cloud documentation about BigQuery, along with citations of the information.

## Grounded Chat Responses

You can also use grounding when working with chat models in Vertex AI. In this example, you'll compare LLM responses with no grounding with responses that are grounded in the results of a Google Search and a data store in Vertex AI Search.

You'll ask a question about Vertex AI and a follow up question about managed datasets in Vertex AI:

In [ ]:
PROMPT = "How can I ground LLM responses in Vertex AI?"
PROMPT_FOLLOWUP = "Is grounding available in Gemini models?"

### Chat session without grounding

Start a chat session and send messages to the LLM with no grounding:

In [ ]:
chat = model.start_chat()

print(f"PROMPT: {PROMPT}")
response = chat.send_message(PROMPT)
print(response.text)

print(f"PROMPT: {PROMPT_FOLLOWUP}")
response = chat.send_message(PROMPT_FOLLOWUP)
print(response.text)

### Chat session grounded in Vertex AI Search results

Now you can add the tool to instruct the chat model to first perform a search within your custom data store, then construct an answer based on the relevant documents:

In [ ]:
chat = model.start_chat()

print(f"PROMPT: {PROMPT}")
response = chat.send_message(
    PROMPT,
    tools=[tool],
)
print(response.text)
print(response.candidates[0].grounding_metadata)

print(f"PROMPT: {PROMPT_FOLLOWUP}")
response = chat.send_message(
    PROMPT_FOLLOWUP,
    tools=[tool],
)
print(response.text)
print(response.candidates[0].grounding_metadata)

Copyright 2024 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License